In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
import pandas as pd
import zipfile
import os
import shutil
import random
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
import torchvision.transforms as T
from torchvision.models import resnet18 ,ResNet18_Weights
from torch import nn 
import torch.nn.functional as F
from torch.optim import SGD ,lr_scheduler
import torch.cuda
import time
from torch.autograd import Variable
from torch.utils.data import random_split
from torch.utils.data import TensorDataset, DataLoader
%matplotlib inline 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

d:\Download\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_path = "../Attachment/Attachment 2/"
data_x = np.array([])
data_y = np.array([])
index = -1
data_limit_for_each_class = 5000
for path in os.listdir(data_path):
    index += 1
    data_count = 0
    for img in os.listdir(data_path + path):
        data_count += 1
        image = cv2.imread(data_path + path + "/" + img)
        data_x = np.append(data_x, image)
        data_y = np.append(data_y, image)
        if data_count >= data_limit_for_each_class:
            break
data_x = np.array(data_x)
data_y = np.array(data_y)
print(data_x.shape)
print(data_y.shape)

In [ ]:
np.savetxt("../Attachment/Attachment 2/data_X.txt",data_x)
np.savetxt("../Attachment/Attachment 2/data_Y.txt",data_y)

In [ ]:
filter_size1 = 3        # Convolutional Layer 1.
num_filters1 = 32
filter_size2 = 3         # Convolutional Layer 2.
num_filters2 = 32
filter_size3 = 3          # Convolutional Layer 3.
num_filters3 = 64
fc_size = 128             # Number of neurons in fully-connected layer.
num_channels = 3
img_size = 128
img_size_flat = img_size * img_size * num_channels
img_shape = (img_size, img_size)
classes = ['apple', 'carambola', 'pear', 'plum', 'tomato']
num_classes = len(classes)
batch_size = 32
validation_size = .16
# how long to wait after validation loss stops improving before terminating training
early_stopping = None  # use None if you don't want to implement early stoping
checkpoint_dir = "models/"

In [ ]:
data_x = torch.tensor(np.array(data_x))       
data_y = torch.tensor(np.array(data_y)).long()    
dataset = TensorDataset(data_x, data_y)
dataset_size = data_x.size(dim = 0)
train_size = int(dataset_size * (1-validation_size))
valid_size = int(dataset_size * validation_size)
train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])
data_loader_train = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
data_loader_valid = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(num_channels, num_filters1, filter_size1, padding=filter_size1//2)
        self.conv2 = nn.Conv2d(num_filters1, num_filters2, filter_size2, padding=filter_size2//2)
        self.conv3 = nn.Conv2d(num_filters2, num_filters3, filter_size3, padding=filter_size3//2)

        # Pooling layer
        self.pool = nn.MaxPool2d(2, 2)

        # Fully connected layers
        # Calculate the size of the flattened layer
        self.fc1_size = self._get_conv_output((num_channels, img_size, img_size))
        self.fc1 = nn.Linear(self.fc1_size, fc_size)
        self.fc2 = nn.Linear(fc_size, num_classes)

        # Regularizer
        self.regularizer = nn.L2Norm(scale=0.04)

    def forward(self, x):
        # Apply convolutions and max pooling
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        # Flatten the output for the dense layer
        x = x.view(-1, self.fc1_size)

        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def _get_conv_output(self, shape):
        with torch.no_grad():
            input = torch.rand(1, *shape)
            output = self._forward_features(input)
            return int(np.prod(output.size()))

    def _forward_features(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        return x

model = ConvNet()
print(model)

In [ ]:
def valid():
    '''
    进行验证，返回模型在验证集上的 accuracy
    '''
    sum_true = 0
    model.eval()
    with torch.no_grad():
        for data in data_loader_valid:
            batch_x, batch_y = data[0].to(device), data[1].to(device)
            y_hat = model(batch_x)
            y_hat = torch.tensor([torch.argmax(_) for _ in y_hat]).to(device)
            sum_true += torch.sum(y_hat == batch_y).float()
        return sum_true / valid_size

In [ ]:
epochs = 10   
lr = 0.01
loss_function = nn.CrossEntropyLoss()                                       # 设置损失函数
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=5e-4)  # 设置优化器


for epoch in range(epochs):
    '''
    完成一个 epoch 的训练
    '''
    sum_true = 0
    sum_loss = 0.0
    max_valid_acc = 0
    model.train()
    index = 0
    total_data = len(data_loader_train)
    for data in data_loader_train:
        # 可视化训练过程
        index += 1
        print('Training batch {}/{}'.format(index,total_data),end='\r')
        # 选取对应批次数据的输入和标签
        batch_x, batch_y = data[0].to(device), data[1].to(device)
        y_hat = model(batch_x)
        loss = loss_function(y_hat, batch_y)

        optimizer.zero_grad()   # 梯度清零
        loss.backward()         # 计算梯度
        optimizer.step()        # 更新参数

        y_hat = torch.tensor([torch.argmax(_) for _ in y_hat]).to(device)
        sum_true += torch.sum(y_hat == batch_y).float()
        sum_loss += loss.item()

    train_acc = sum_true / train_size
    train_loss = sum_loss / train_size

    valid_acc = valid()
    if valid_acc > max_valid_acc:
        torch.save(model, "checkpoint.pt")

    print(f"epoch: {epoch}, train loss: {train_loss:.4f}, train accuracy: {train_acc*100:.2f}%, valid accuracy: {valid_acc*100:.2f}%, time: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()) }")
